In [1]:
!pip install emoji

     |████████████████████████████████| 51kB 4.4MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=3ffbc8040815eabb1f87a6b1ff40a37aeab96b077dc26ea0d8324f7cdd9b74ce
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [2]:
import numpy as np
import pandas as pd

In [3]:
import emoji

In [4]:
emoji_dictionary = {
    "0" : "\u2764\uFE0F", #prints a red heart
    "1" : ":baseball:",
    "2" : ":grinning_face_with_big_eyes:",
    "3" : ":disappointed_face:",
    "4" : ":fork_and_knife:",
    "5" : ":hundred_points:",
    "6" : ":fire:",
    "7" : ":face_blowing_a_kiss:",
    "8" : ":chestnut:",
    "9" : ":flexed_biceps:"
}

In [5]:
for e in emoji_dictionary.values():
  print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [6]:
from google.colab import files 
uploaded = files.upload()

Saving train_emoji.csv to train_emoji.csv


In [7]:
from google.colab import files 
uploaded = files.upload()

Saving test_emoji.csv to test_emoji.csv


In [8]:
train = pd.read_csv("train_emoji.csv",header=None)
test = pd.read_csv("test_emoji.csv",header=None)

In [9]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [10]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😃
The assignment is too long  😞
I want to go play ⚾


In [11]:
from keras.utils import to_categorical

In [12]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)


(132,)
(56,)
(132, 5)
(56, 5)


In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-26 10:12:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-26 10:12:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-26 10:12:32--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [14]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [15]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [16]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [17]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [19]:

from keras.layers import *
from keras.models import Sequential

In [20]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=64,epochs=150,shuffle=True,validation_split=0.2)

Epoch 1/150
2/2 [==============================] - 1s 347ms/step - loss: 1.6150 - acc: 0.2000 - val_loss: 1.6161 - val_acc: 0.2593
Epoch 2/150
2/2 [==============================] - 0s 13ms/step - loss: 1.5690 - acc: 0.2381 - val_loss: 1.6188 - val_acc: 0.2222
Epoch 3/150
2/2 [==============================] - 0s 12ms/step - loss: 1.5234 - acc: 0.3143 - val_loss: 1.6338 - val_acc: 0.2222
Epoch 4/150
2/2 [==============================] - 0s 12ms/step - loss: 1.5023 - acc: 0.3714 - val_loss: 1.6630 - val_acc: 0.2222
Epoch 5/150
2/2 [==============================] - 0s 12ms/step - loss: 1.4897 - acc: 0.3429 - val_loss: 1.6963 - val_acc: 0.2222
Epoch 6/150
2/2 [==============================] - 0s 13ms/step - loss: 1.4390 - acc: 0.4381 - val_loss: 1.7153 - val_acc: 0.2222
Epoch 7/150
2/2 [==============================] - 0s 18ms/step - loss: 1.4485 - acc: 0.3905 - val_loss: 1.7186 - val_acc: 0.2222
Epoch 8/150
2/2 [==============================] - 0s 13ms/step - loss: 1.4057 - acc: 0.4

In [22]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 4ms/step - loss: 2.4268 - acc: 0.5714


[2.4268290996551514, 0.5714285969734192]

In [23]:
pred = model.predict_classes(emb_Xt)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [24]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😃
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😃
😞
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
